# Data Preparation

This Notebook opens all csv files and stores them in a concatenated dataframe.

## Features Extraction & Features Engineering

I won't work with all features for modeling process. 

### Features Extrcation

Only relevant future will be store in the training/validation set. Those data are in 3 categories:

- static data: team name, referee, game details
- past values (goals, 1/N/2 for last x games)
- odds by betting platform

### Features Engineering

Some features will be created to create new one.

# Final Model & Predictions

I want that the user just fill a few information for the predictions.
The user must gives:

- the game (Home Team / Away Team)
- the date (which will converted in "moment" of the season)
- the odds he get from a betting platform
- team trends = a feeling (his/her own opinion of the team based on past 5 games)

The model will be trained with a computed value for the "trends" based on last 5 games scores.

An observation: Liverpool - Tottenham has odds (1/N2) 1.57 / 3.05 / 3.85 (THIS VALUES COME FROM THE BETTING WEBSITE)

The observation in the model will have the same pattern, but odds will be computed based on the past games values.


In [5]:
import pyforest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

import os

In [7]:
# create a list of files
files_and_directories = os.listdir("../data/")
files_and_directories

# read all files
# store 
all_files = []
for file in files_and_directories:
    if 'csv' in file:
        temp = pd.read_csv('../data/'+str(file))
        all_files.append(temp)
        
df = pd.concat(all_files, sort=False)

# print the size of the full dataframe
df.shape

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(7610, 80)

In [8]:
df.tail()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,SJA,BSH,BSD,BSA,SBH,SBD,SBA,0,1,2
375,E0,2010-05-09,Everton,Portsmouth,1.0,0.0,H,0.0,0.0,D,...,11.00,1.30,5.00,10.00,1.29,5.00,8.50,NaN,NaN,NaN
376,E0,2010-05-09,Hull,Liverpool,0.0,0.0,D,0.0,0.0,D,...,1.57,6.00,3.75,1.57,6.00,3.75,1.50,NaN,NaN,NaN
377,E0,2010-05-09,Man United,Stoke,4.0,0.0,H,2.0,0.0,H,...,21.00,1.11,7.50,23.00,1.11,7.50,15.00,NaN,NaN,NaN
378,E0,2010-05-09,West Ham,Man City,1.0,1.0,D,1.0,1.0,D,...,1.91,4.20,3.60,1.80,4.00,3.40,1.80,NaN,NaN,NaN
379,E0,2010-05-09,Wolves,Sunderland,2.0,1.0,H,1.0,1.0,D,...,3.00,2.38,3.25,3.00,2.25,3.20,2.88,NaN,NaN,NaN


In [9]:
# convert date columns
df['Date'] = pd.to_datetime(df['Date'])

# delete columns with all NaN
df.dropna(axis=1, how='all', inplace=True)
print(df.shape)

<IPython.core.display.Javascript object>

(7610, 80)


In [10]:
missing_columns = pd.DataFrame(df.isnull().sum())
missing_columns.columns = ['Cnt']
missing_columns.head()

<IPython.core.display.Javascript object>

,Cnt
Div,10
Date,10
HomeTeam,10
AwayTeam,10
FTHG,10


In [11]:
for idx, r in missing_columns.iterrows():
    if r['Cnt']>30:
        df.drop(idx,axis=1,inplace=True)
        
df.shape

(7610, 56)

## Explory Data Analysis

- append values (SimpleImputer)
- convert some columns
- plot some descriptive statistics


In [12]:
# transform OHE the FTR (full time 1/N/2) and the Half Time (1/N/2)
df.reset_index(inplace=True, drop=True)
temp_ohe = pd.get_dummies(df[['FTR','HTR']])
df = df.merge(temp_ohe, left_index=True, right_index=True)
df.drop(['Div'], axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [13]:
# encode the refereee
imp = SimpleImputer(strategy="most_frequent")
referee = imp.fit_transform(df[['Referee']])
df['Referee']=referee
oe = OrdinalEncoder()
referee = oe.fit_transform(df[['Referee']])

df['Referee']=referee

df.shape

(7610, 61)

In [14]:
df.dropna(axis=0, inplace=True)

In [15]:
df.isnull().sum().sum()

0

In [16]:
df.shape

(7576, 61)

In [17]:
df.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
       'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2',
       'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU',
       'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh',
       'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'FTR_A', 'FTR_D', 'FTR_H',
       'HTR_A', 'HTR_D', 'HTR_H'],
      dtype='object')

# Data definition
http://www.football-data.co.uk/notes.txt

### Key to results data:

- Div = League Division
- Date = Match Date (dd/mm/yy)
- Time = Time of match kick off
- HomeTeam = Home Team
- AwayTeam = Away Team
- FTHG and HG = Full Time Home Team Goals
- FTAG and AG = Full Time Away Team Goals
- FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
- HTHG = Half Time Home Team Goals
- HTAG = Half Time Away Team Goals
- HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

### Match Statistics (where available)
- Attendance = Crowd Attendance
- Referee = Match Referee
- HS = Home Team Shots
- AS = Away Team Shots
- HST = Home Team Shots on Target
- AST = Away Team Shots on Target
- HHW = Home Team Hit Woodwork
- AHW = Away Team Hit Woodwork
- HC = Home Team Corners
- AC = Away Team Corners
- HF = Home Team Fouls Committed
- AF = Away Team Fouls Committed
- HFKC = Home Team Free Kicks Conceded
- AFKC = Away Team Free Kicks Conceded
- HO = Home Team Offsides
- AO = Away Team Offsides
- HY = Home Team Yellow Cards
- AY = Away Team Yellow Cards
- HR = Home Team Red Cards
- AR = Away Team Red Cards
- HBP = Home Team Bookings Points (10 = yellow, 25 = red)
- ABP = Away Team Bookings Points (10 = yellow, 25 = red)

### Key to 1X2 (match) betting odds data:

- B365H = Bet365 home win odds
- B365D = Bet365 draw odds
- B365A = Bet365 away win odds
- BSH = Blue Square home win odds
- BSD = Blue Square draw odds
- BSA = Blue Square away win odds
- BWH = Bet&Win home win odds
- BWD = Bet&Win draw odds
- BWA = Bet&Win away win odds
- GBH = Gamebookers home win odds
- GBD = Gamebookers draw odds
- GBA = Gamebookers away win odds
- IWH = Interwetten home win odds
- IWD = Interwetten draw odds
- IWA = Interwetten away win odds
- LBH = Ladbrokes home win odds
- LBD = Ladbrokes draw odds
- LBA = Ladbrokes away win odds
- PSH and PH = Pinnacle home win odds
- PSD and PD = Pinnacle draw odds
- PSA and PA = Pinnacle away win odds
- SOH = Sporting Odds home win odds
- SOD = Sporting Odds draw odds
- SOA = Sporting Odds away win odds
- SBH = Sportingbet home win odds
- SBD = Sportingbet draw odds
- SBA = Sportingbet away win odds
- SJH = Stan James home win odds
- SJD = Stan James draw odds
- SJA = Stan James away win odds
- SYH = Stanleybet home win odds
- SYD = Stanleybet draw odds
- SYA = Stanleybet away win odds
- VCH = VC Bet home win odds
- VCD = VC Bet draw odds
- VCA = VC Bet away win odds
- WHH = William Hill home win odds
- WHD = William Hill draw odds
- WHA = William Hill away win odds

## Explore the betting columns

The target is to build set of pronostics [x.yy, x.yy, x.yy] which means Home, D, Away for each betting platform.

Missing value will be filled with the line (other betting platform).

Betting score are always in 3 features which ended by:
- "H" for home
- "D" for deuce
- "A" for Away

### Extract columns from the dataset

Extract columns from dataset and build a dictionary of feature name by betting platform


In [22]:
all_betting_cols = [x for x in df.columns.tolist() if str(x).endswith("H") 
                    or str(x).endswith("D") or str(x).endswith("A")]
all_betting_cols

# from that list extract the betting platform
betting_platform={}
for v in all_betting_cols:
    bp = v[:-1]
    if bp != 'FTR_' and bp != 'HTR_' and bp != 'BbA' and bp != 'BbMxAH' and bp != 'BbAvAH':
        betting_platform[bp] = [f"{bp}H",f"{bp}D",f"{bp}A"]

betting_platform

{'B365': ['B365H', 'B365D', 'B365A'],
 'BW': ['BWH', 'BWD', 'BWA'],
 'IW': ['IWH', 'IWD', 'IWA'],
 'WH': ['WHH', 'WHD', 'WHA'],
 'VC': ['VCH', 'VCD', 'VCA'],
 'BbMx': ['BbMxH', 'BbMxD', 'BbMxA'],
 'BbAv': ['BbAvH', 'BbAvD', 'BbAvA']}

### create the new dataset
read each row of the original dataset and for the betting platform create a new row, the dictionary will be
- day features such as teams and score
- date
- betting platform odds


In [44]:
new_set = []
for idx, r in df.iterrows():
    static_variable = [r.HomeTeam, r.AwayTeam, r.Date.month]
    for key in betting_platform:
        temp = betting_platform[key]
        temp_odds =r[temp]
        new_set.append([static_variable[0],
                        static_variable[1],
                        static_variable[2],
                        temp_odds[0], temp_odds[1], temp_odds[2],
                        r.FTR]
                        )


In [45]:
bet=pd.DataFrame(new_set)
bet.dropna(inplace=True)
bet.shape

<IPython.core.display.Javascript object>

(53032, 7)

In [46]:
bet.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

In [47]:
bet.dtypes

0     object
1     object
2      int64
3    float64
4    float64
5    float64
6     object
dtype: object

In [48]:
bet.sample(6)

,0,1,2,3,4,5,6
6154,Aston Villa,Arsenal,11,5.00,3.80,1.65,D
36990,Bournemouth,Burnley,5,2.00,3.40,3.80,H
28958,Liverpool,Stoke,4,1.31,5.54,9.73,D
38802,Wigan,Everton,4,3.60,3.30,2.05,D
22552,Huddersfield,Stoke,12,2.57,3.30,3.25,D
38502,Wolves,Chelsea,2,5.40,3.60,1.57,A


In [49]:
#bet.loc[:,5] = bet.loc[:,5].map({'H':-1,'D':0,'A':1})
bet.sample(9)

,0,1,2,3,4,5,6
16522,Fulham,Man United,2,5.60,3.90,1.55,A
51086,Portsmouth,Wigan,10,2.38,3.30,3.00,H
3353,Arsenal,Fulham,10,1.53,4.20,6.00,D
35798,Swansea,Bournemouth,12,2.90,3.40,2.60,A
12027,Southampton,Man City,12,9.50,5.00,1.34,A
36867,Everton,Chelsea,4,3.90,3.70,2.10,A
45526,Southampton,QPR,9,1.51,4.80,8.90,H
49921,Sunderland,West Ham,3,2.25,3.40,3.50,A
7258,Swansea,Newcastle,2,2.08,3.41,3.54,H


In [50]:
oe = OrdinalEncoder()
bet['H']=oe.fit_transform(np.array(bet.loc[:,0].tolist()).reshape(-1,1))
bet['A']=oe.fit_transform(np.array(bet.loc[:,1].tolist()).reshape(-1,1))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
bet.columns=['Hteam','Ateam','Month','1','N','2','Result','TeamH','TeamA']
bet.sample(9)

,Hteam,Ateam,Month,1,N,2,Result,TeamH,TeamA
3336,Liverpool,Newcastle,4,1.57,4.20,6.5,D,17.0,21.0
32822,Hull,West Brom,6,2.39,3.22,3.1,D,15.0,32.0
27401,Swansea,Bournemouth,11,2.80,3.20,2.7,D,29.0,6.0
9293,Leicester,Man City,12,4.40,4.10,1.8,D,16.0,18.0
8715,Watford,West Ham,10,2.60,3.40,2.9,H,31.0,33.0
12174,Arsenal,Chelsea,1,3.00,3.50,2.3,H,0.0,10.0
42604,Man City,Liverpool,8,2.40,3.20,2.7,H,18.0,17.0
47743,Everton,Tottenham,5,2.40,3.10,3.1,A,12.0,30.0
12421,Liverpool,Bournemouth,9,1.20,7.00,15.0,H,17.0,6.0


In [54]:
bet.to_csv('../data/bet.csv', index=False)